In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['X_test.npy', 'X_train.npy', 'sample_submission.csv', 'y_train.npy']


In [2]:
import pandas as pd
import numpy as np
data = np.load('../input/X_train.npy')
labels = np.load('../input/y_train.npy')
print(labels.shape,data.shape)

(10000,) (10000, 28, 28)


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25)

In [4]:
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)
print(X_train.shape)
print(y_train[:10])

(7500, 784)
[7 0 7 2 5 1 9 0 5 7]


In [5]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='saga',multi_class ='multinomial')
clf.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
def evaluate(pred,labels):
    return sum(pred == labels)/len(pred)

In [7]:
pred = clf.predict(X_test)
print("Accuracy:",evaluate(pred,y_test))

Accuracy: 0.256


In [8]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=10)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [9]:
pred = rfc.predict(X_test)
print("Accuracy:",evaluate(pred,y_test))

Accuracy: 0.9348


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

def cat_to_onehot(labels):
    labels_onehot = np.zeros((len(labels),10),dtype=np.int)
    for i in range(len(labels)):
        labels_onehot[i][labels[i]] = 1
    return labels_onehot

def onehot_to_cat(labels_onehot):
    labels = labels_onehot.argmax(axis=1)
    return labels

X_train = X_train/255
X_test = X_test/255

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=(28*28)))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

y_train_onehot = cat_to_onehot(y_train)
y_test_onehot = cat_to_onehot(y_test)
print(X_train.shape,y_train.shape)
model.fit(X_train,y_train_onehot,epochs=10)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                170       
Total params: 25,818
Trainable params: 25,818
Non-trainable params: 0
_________________________________________________________________
None
(7500, 784) (7500,)
Epoch 1/10
7500/7500 [==============================] - 1s 132us/step - loss: 2.2092 - acc: 0.1671
Epoch 2/10
7500/7500 [==============================] - 0s 57us/step - loss: 1.7719 - acc: 0.3632
Epoch 3/10
7500/7500 [==============================] - 0s 52us/step - loss: 1.2590 - acc: 0.5731
Epoch 4/10
7500/7500 [==============================] - 0s 49us/step - loss: 0.904

In [11]:
pred_onehot = model.predict(X_test)
evaluate(y_test,onehot_to_cat(pred_onehot))

0.8936

In [12]:
unknown = np.load("../input/X_test.npy")
unknown = unknown.reshape(unknown.shape[0],-1)

pred = rfc.predict(unknown)

In [13]:
out = pd.DataFrame(columns=['Id','Prediction'])
out['Id'] = range(len(pred))
out['Prediction'] = pred
print(out.info())
out.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 2 columns):
Id            90000 non-null int64
Prediction    90000 non-null uint8
dtypes: int64(1), uint8(1)
memory usage: 791.1 KB
None


,Id,Prediction
0,0,8
1,1,8
2,2,7
3,3,3
4,4,6
5,5,6
6,6,4
7,7,0
8,8,2
9,9,3


In [14]:
out.to_csv('./out.csv',index=False)